<a href="https://colab.research.google.com/github/pranjal299/cv-project/blob/main/Weight_Estimation_ML_Image_Descriptors_Cropped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure you have this folder as shortcut in your drive - /content/gdrive/MyDrive/Morota-video-data/ 

In [1]:
#Mount Google Drive 
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


### Mapping Table Input

In [100]:
import tensorflow as tf
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt


final_map = pd.read_csv("/content/gdrive/MyDrive/Morota-video-data/final_mapping.csv")
weeks = final_map
weeks

,image_path,Date,Pig,weight,Week,pig_on_edge,width,length,height,height_adjusted
0,Week1/20210922/20210922_3330_b/_Depth_4683.jpg,20210922,3330,15.5,1,0,42,105,36,219
1,Week1/20210922/20210922_3330_b/_Depth_4682.jpg,20210922,3330,15.5,1,0,44,115,36,219
2,Week1/20210922/20210922_3330_b/_Depth_4625.jpg,20210922,3330,15.5,1,0,59,169,38,217
3,Week1/20210922/20210922_3330_b/_Depth_4704.jpg,20210922,3330,15.5,1,0,1,1,47,208
4,Week1/20210922/20210922_3330_b/_Depth_4855.jpg,20210922,3330,15.5,1,0,148,70,33,222
...,...,...,...,...,...,...,...,...,...,...
36587,Week10/20211119/20211119_3342/_Depth_3044.jpg,20211119,3342,56.6,10,0,5,12,60,195
36588,Week10/20211119/20211119_3342/_Depth_2546.jpg,20211119,3342,56.6,10,0,193,82,49,206
36589,Week10/20211119/20211119_3342/_Depth_2741.jpg,20211119,3342,56.6,10,0,76,233,52,203
36590,Week10/20211119/20211119_3342/_Depth_2754.jpg,20211119,3342,56.6,10,0,205,67,42,213


In [101]:
#weeks = weeks[(weeks['width']>45) & (weeks['length']>45) & (weeks['height']>45)]

In [104]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

print('Linear Regression')
print()

# Load the dataset
data = weeks.copy()

mae_all = 0
mape_all = 0
for test_pig in [3384,3385,3330,3342]:
  
  print('Test Pig - ', test_pig)

  # Split the dataset into train and test sets
  train_data = data[data['Pig'] != test_pig]
  test_data = data[data['Pig'] == test_pig]
  
  # Separate the features (width, length, height) and target (weight) variables
  X_train = train_data[['width', 'length', 'height_adjusted']]
  y_train = train_data['weight']
  X_test = test_data[['width', 'length', 'height_adjusted']]
  y_test = test_data['weight']

  # Create a Linear Regression model
  model = LinearRegression()

  # Train the model
  model.fit(X_train, y_train)

  # Make predictions on the test set
  y_pred = model.predict(X_test)

  # Calculate the mean absolute error (MAE)
  mae = np.mean(np.abs(y_test - y_pred))
  print(f'Mean Absolute Error: {mae:.2f}')

  # Calculate the mean absolute percentage error (MAPE)
  mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
  print(f'Mean Absolute Percentage Error: {mape:.2f}%')
  
  mae_all += mae
  mape_all += mape

  print('--------------------------------------------------------------------------')

print()
print('Average MAE: ', mae_all/4)
print('Average MAPE: ', mape_all/4, '%')

Linear Regression

Test Pig -  3384
Mean Absolute Error: 6.16
Mean Absolute Percentage Error: 26.47%
--------------------------------------------------------------------------
Test Pig -  3385
Mean Absolute Error: 8.47
Mean Absolute Percentage Error: 40.32%
--------------------------------------------------------------------------
Test Pig -  3330
Mean Absolute Error: 9.52
Mean Absolute Percentage Error: 27.21%
--------------------------------------------------------------------------
Test Pig -  3342
Mean Absolute Error: 10.00
Mean Absolute Percentage Error: 24.59%
--------------------------------------------------------------------------

Average MAE:  8.537767019322304
Average MAPE:  29.64606291399899 %


In [105]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

print('Random Forest')
print()

# Load the dataset
data = weeks.copy()

mae_all = 0
mape_all = 0
for test_pig in [3384, 3385, 3330, 3342]:
  
    print('Test Pig - ', test_pig)

    # Split the dataset into train and test sets
    train_data = data[data['Pig'] != test_pig]
    test_data = data[data['Pig'] == test_pig]

    # Separate the features (width, length, height) and target (weight) variables
    X_train = train_data[['width', 'length', 'height_adjusted']]
    y_train = train_data['weight']
    X_test = test_data[['width', 'length', 'height_adjusted']]
    y_test = test_data['weight']

    # Apply feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Create a RandomForestRegressor model
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    model.fit(X_train_scaled, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test_scaled)

    # Calculate the mean absolute error (MAE)
    mae = np.mean(np.abs(y_test - y_pred))
    print(f'Mean Absolute Error: {mae:.2f}')

    # Calculate the mean absolute percentage error (MAPE)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f'Mean Absolute Percentage Error: {mape:.2f}%')

    mae_all += mae
    mape_all += mape

    print('--------------------------------------------------------------------------')

print()
print('Average MAE: ', mae_all/4)
print('Average MAPE: ', mape_all/4, '%')


Random Forest

Test Pig -  3384
Mean Absolute Error: 6.97
Mean Absolute Percentage Error: 27.48%
--------------------------------------------------------------------------
Test Pig -  3385
Mean Absolute Error: 6.49
Mean Absolute Percentage Error: 29.61%
--------------------------------------------------------------------------
Test Pig -  3330
Mean Absolute Error: 7.88
Mean Absolute Percentage Error: 22.43%
--------------------------------------------------------------------------
Test Pig -  3342
Mean Absolute Error: 8.27
Mean Absolute Percentage Error: 21.62%
--------------------------------------------------------------------------

Average MAE:  7.403060262523626
Average MAPE:  25.287353585676545 %


In [107]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# Load the dataset
data = weeks.copy()

models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=0.1),
    'RandomForestRegressor': RandomForestRegressor(n_estimators=100, random_state=42),
    'GradientBoostingRegressor': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'SVR': SVR(kernel='rbf', C=1, gamma='scale'),
    'ExtraTreesRegressor': ExtraTreesRegressor(n_estimators=100, random_state=42)
}

for model_name, model in models.items():
    print(f'Using {model_name}')

    mae_all = 0
    mape_all = 0
    for test_pig in [3384, 3385, 3330, 3342]:

        print('Test Pig - ', test_pig)

        # Split the dataset into train and test sets
        train_data = data[data['Pig'] != test_pig]
        test_data = data[data['Pig'] == test_pig]

        # Separate the features (width, length, height) and target (weight) variables
        X_train = train_data[['width', 'length', 'height_adjusted']]
        y_train = train_data['weight']
        X_test = test_data[['width', 'length', 'height_adjusted']]
        y_test = test_data['weight']

        # Apply feature scaling
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Train the model
        model.fit(X_train_scaled, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test_scaled)

        # Calculate the mean absolute error (MAE)
        mae = np.mean(np.abs(y_test - y_pred))
        print(f'Mean Absolute Error: {mae:.2f}')

        # Calculate the mean absolute percentage error (MAPE)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        print(f'Mean Absolute Percentage Error: {mape:.2f}%')

        mae_all += mae
        mape_all += mape

        print('--------------------------------------------------------------------------')

    print()
    print('Average MAE: ', mae_all/4)
    print('Average MAPE: ', mape_all/4, '%')
    print('==========================================================================')
    print()
    print()


Using LinearRegression
Test Pig -  3384
Mean Absolute Error: 6.16
Mean Absolute Percentage Error: 26.47%
--------------------------------------------------------------------------
Test Pig -  3385
Mean Absolute Error: 8.47
Mean Absolute Percentage Error: 40.32%
--------------------------------------------------------------------------
Test Pig -  3330
Mean Absolute Error: 9.52
Mean Absolute Percentage Error: 27.21%
--------------------------------------------------------------------------
Test Pig -  3342
Mean Absolute Error: 10.00
Mean Absolute Percentage Error: 24.59%
--------------------------------------------------------------------------

Average MAE:  8.537767019322304
Average MAPE:  29.646062913998993 %


Using Ridge
Test Pig -  3384
Mean Absolute Error: 6.16
Mean Absolute Percentage Error: 26.47%
--------------------------------------------------------------------------
Test Pig -  3385
Mean Absolute Error: 8.47
Mean Absolute Percentage Error: 40.32%
--------------------------

In [108]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Build a simple feed-forward neural network
def build_model(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_absolute_error', optimizer=Adam(lr=0.001))
    return model

# Create the neural network model
nn_model = build_model((3,))

# Train and evaluate the neural network model
mae_all = 0
mape_all = 0
for test_pig in [3384, 3385, 3330, 3342]:

    print('Test Pig - ', test_pig)

    # Split the dataset into train and test sets
    train_data = data[data['Pig'] != test_pig]
    test_data = data[data['Pig'] == test_pig]

    # Separate the features (width, length, height) and target (weight) variables
    X_train = train_data[['width', 'length', 'height_adjusted']]
    y_train = train_data['weight']
    X_test = test_data[['width', 'length', 'height_adjusted']]
    y_test = test_data['weight']

    # Apply feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train the model
    nn_model.fit(X_train_scaled, y_train, epochs=100, batch_size=16, verbose=0)

    # Make predictions on the test set
    y_pred = nn_model.predict(X_test_scaled).squeeze()

    # Calculate the mean absolute error (MAE)
    mae = np.mean(np.abs(y_test - y_pred))
    print(f'Mean Absolute Error: {mae:.2f}')

    # Calculate the mean absolute percentage error (MAPE)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    print(f'Mean Absolute Percentage Error: {mape:.2f}%')

    mae_all += mae
    mape_all += mape

    print('--------------------------------------------------------------------------')

print()
print('Average MAE: ', mae_all/4)
print('Average MAPE: ', mape_all/4, '%')


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Test Pig -  3384
291/291 [==============================] - 1s 1ms/step
Mean Absolute Error: 5.42
Mean Absolute Percentage Error: 21.16%
--------------------------------------------------------------------------
Test Pig -  3385
209/209 [==============================] - 0s 2ms/step
Mean Absolute Error: 4.10
Mean Absolute Percentage Error: 18.69%
--------------------------------------------------------------------------
Test Pig -  3330
287/287 [==============================] - 0s 2ms/step
Mean Absolute Error: 7.96
Mean Absolute Percentage Error: 21.10%
--------------------------------------------------------------------------
Test Pig -  3342
359/359 [==============================] - 1s 1ms/step
Mean Absolute Error: 9.09
Mean Absolute Percentage Error: 22.32%
--------------------------------------------------------------------------

Average MAE:  6.642610181519008
Average MAPE:  20.81471998325042 %


In [ ]:
from sklearn.neural_network import MLPRegressor

models = {
    'MLP Regressor': MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000, random_state=42),
}

for model_name, model in models.items():

    print(f'==== {model_name} ====')

    mae_all = 0
    mape_all = 0
    for test_pig in [3384, 3385, 3330, 3342]:

        print('Test Pig - ', test_pig)

        # Split the dataset into train and test sets
        train_data = data[data['Pig'] != test_pig]
        test_data = data[data['Pig'] == test_pig]

        # Separate the features (width, length, height) and target (weight) variables
        X_train = train_data[['width', 'length', 'height_adjusted']]
        y_train = train_data['weight']
        X_test = test_data[['width', 'length', 'height_adjusted']]
        y_test = test_data['weight']

        # Apply feature scaling
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Train the model
        model.fit(X_train_scaled, y_train)

        # Make predictions on the test set
        y_pred = model.predict(X_test_scaled)

        # Calculate the mean absolute error (MAE)
        mae = np.mean(np.abs(y_test - y_pred))
        print(f'Mean Absolute Error: {mae:.2f}')

        # Calculate the mean absolute percentage error (MAPE)
        mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
        print(f'Mean Absolute Percentage Error: {mape:.2f}%')

        mae_all += mae
        mape_all += mape

        print('--------------------------------------------------------------------------')

    print()
    print('Average MAE: ', mae_all/4)
    print('Average MAPE: ', mape_all/4, '%')
    print()


==== MLP Regressor ====
Test Pig -  3384
Mean Absolute Error: 6.63
Mean Absolute Percentage Error: 26.36%
--------------------------------------------------------------------------
Test Pig -  3385
